# Quick Start

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [5]:
# download training data
# Download training data from open datasets.
train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [7]:
# create data loaders
batch_size = 64
train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(X.shape)
    print(y.shape, y.dtype)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64]) torch.int64


In [12]:
# creating models
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = MLP()
print(model)

MLP(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [14]:
# optimize model parameters
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [18]:
def train(dataloader, model, loss, optimizer):
    size = len(dataloader.dataset)    
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [19]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [20]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.296008  [    0/60000]
loss: 2.292178  [ 6400/60000]
loss: 2.283893  [12800/60000]
loss: 2.291763  [19200/60000]
loss: 2.283356  [25600/60000]
loss: 2.242780  [32000/60000]
loss: 2.268028  [38400/60000]
loss: 2.229088  [44800/60000]
loss: 2.225590  [51200/60000]
loss: 2.235595  [57600/60000]
Test Error: 
 Accuracy: 38.7%, Avg loss: 2.216909 

Epoch 2
-------------------------------
loss: 2.209920  [    0/60000]
loss: 2.211812  [ 6400/60000]
loss: 2.176525  [12800/60000]
loss: 2.209809  [19200/60000]
loss: 2.164269  [25600/60000]
loss: 2.101141  [32000/60000]
loss: 2.169339  [38400/60000]
loss: 2.080078  [44800/60000]
loss: 2.091931  [51200/60000]
loss: 2.116673  [57600/60000]
Test Error: 
 Accuracy: 47.5%, Avg loss: 2.070608 

Epoch 3
-------------------------------
loss: 2.075043  [    0/60000]
loss: 2.067087  [ 6400/60000]
loss: 1.989468  [12800/60000]
loss: 2.061678  [19200/60000]
loss: 1.954165  [25600/60000]
loss: 1.881151  [32000/600

In [21]:
# saving and loading models
torch.save(model.state_dict(), 'model.pth')

In [22]:
model = MLP()
model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [24]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
